In [75]:
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
import numpy as np

# read datasets
train_df = pd.read_csv('data/train3.csv')
test_df = pd.read_csv('data/test3.csv')
val_df = pd.read_csv('data/validate3.csv')

y1_name, y2_name, y3_name = "dir_costs", "traffic_costs_s_r", "lost_trips_costs_s_r"
train_y1, train_y2, train_y3 = train_df[y1_name], train_df[y2_name], train_df[y3_name]
test_y1, test_y2, test_y3 = test_df[y1_name], test_df[y2_name], test_df[y3_name]
val_y1, val_y2, val_y3 = val_df[y1_name], val_df[y2_name], val_df[y3_name]


# scale features
X_train = train_df.drop(columns=[y1_name, y2_name, y3_name])
scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_test = test_df.drop(columns=[y1_name, y2_name, y3_name])
X_test = scaler.transform(X_test)

X_val = val_df.drop(columns=[y1_name, y2_name, y3_name])
X_val = scaler.transform(X_val)

# prepare dataset from training kmeans
X_train_y1 = np.concatenate((X_train, train_df[y1_name].values.reshape(-1, 1)), axis=1)
scaler_y1 = MaxAbsScaler()
scaler_y1.fit(X_train_y1)
X_train_y1 = scaler_y1.transform(X_train_y1)

<h1> Pick optimal number of clusters using silhouette score </h1>

In [76]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Assuming 'X' is your dataset

# Initialize an empty list to store silhouette scores
silhouette_scores = []

# Iterate over different numbers of clusters
for n_clusters in range(2, 11):
    # Initialize KMeans clustering algorithm with 'n_clusters'
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    
    # Fit the algorithm to the data
    kmeans.fit(X_train_y1)
    
    # Compute the silhouette score for the current number of clusters
    silhouette_avg = silhouette_score(X_train_y1, kmeans.labels_)
    
    # Append the silhouette score to the list
    silhouette_scores.append(silhouette_avg)

# Choose the optimal number of clusters based on the maximum silhouette score
optimal_n_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
print("Optimal number of clusters:", optimal_n_clusters)

Optimal number of clusters: 2


<h1> Train kmeans with Yi </h1>

In [77]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=42, n_init=10)
kmeans.fit(X_train_y1)

KMeans(n_clusters=2, n_init=10, random_state=42)

In [78]:
import numpy as np
# transforms X into clustered dataset with. k - number of clusters in the range [0, k-1]
def transform_to_clustered_dataset_train(X: np.ndarray, Xy: pd.DataFrame, cluster_predictor, k: int):
    clusters = cluster_predictor.predict(Xy)
    zero_columns = pd.DataFrame(np.zeros((X.shape[0], k)), columns=[f'class_{i}' for i in range(k)])
    res = np.concatenate((X, zero_columns), axis=1)
    for i in range(res.shape[0]):
        cluster = clusters[i]
        res[i, X.shape[1] + cluster] = 1.0 
    return res

def transform_to_clustered_dataset_test(X: np.ndarray, kmeans_model: KMeans, k: int):
    centers = kmeans_model.cluster_centers_[:, :-1]
    clusters = []
    for row in X:
        min_dist = np.Infinity
        cluster = -1
        for i in range(k):
            cur_dist = np.linalg.norm(row - centers[i])
            if cur_dist < min_dist:
                min_dist = cur_dist
                cluster = i
        if cluster == -1:
            raise "Error"
        clusters.append(cluster)
        
    zero_columns = pd.DataFrame(np.zeros((X.shape[0], k)), columns=[f'class_{i}' for i in range(k)])
    res = np.concatenate((X, zero_columns), axis=1)
    for i in range(res.shape[0]):
        cluster = clusters[i]
        res[i, X.shape[1] + cluster] = 1.0 
    return res

In [79]:
X_train_clustered = transform_to_clustered_dataset_train(X_train, X_train_y1, kmeans, optimal_n_clusters)
X_test_clustered = transform_to_clustered_dataset_test(X_test, kmeans, optimal_n_clusters)
X_val_clustered = transform_to_clustered_dataset_test(X_val, kmeans, optimal_n_clusters)

<h1> GBR with arbitrary hyperparameters </h1>

In [80]:
from metrics import print_metrics
from sklearn.ensemble import GradientBoostingRegressor

# Create an instance of GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor(n_estimators=48, learning_rate=0.096952, max_depth=6, random_state=42)

# Fit the regressor to the training data
gb_regressor.fit(X_train_clustered, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, gb_regressor.predict(X_test_clustered))

print("------ train metrics ------")
print_metrics(train_y1, gb_regressor.predict(X_train_clustered))

print("------ validate metrics ------")
print_metrics(val_y1, gb_regressor.predict(X_val_clustered))


------ test metrics ------
Mean Squared Error (MSE):              25413220988754.0507812500
Root Mean Squared Error (RMSE):        5041152.7440411933
Mean Absolute Error (MAE):             2907404.9537322405
R-squared (R²):                        0.8416290943
Mean Absolute Percentage Error (MAPE): 0.0714822692
Max Error (ME):                        28277735.2133217528
Median Absolute Error (MedAE):         1487218.6317628622
------ train metrics ------
Mean Squared Error (MSE):              4836709895294.2695312500
Root Mean Squared Error (RMSE):        2199252.1218119287
Mean Absolute Error (MAE):             1297865.9018217572
R-squared (R²):                        0.9726575355
Mean Absolute Percentage Error (MAPE): 0.0334095967
Max Error (ME):                        14103739.0102937371
Median Absolute Error (MedAE):         701405.1215910893
------ validate metrics ------
Mean Squared Error (MSE):              35855809016862.7109375000
Root Mean Squared Error (RMSE):        5987972.

<h1> Optimize Gradient boost parameters using Differential evolution</h1>

In [81]:
from sklearn.metrics import r2_score

# define objective function
def objective_function(params, train_X, train_y, test_X, test_y):
    regressor = GradientBoostingRegressor(n_estimators=int(params[0]), learning_rate=params[1], max_depth=int(params[2]), random_state=42)
    regressor.fit(train_X, train_y)
    pred_y = regressor.predict(test_X)
    r2 = r2_score(test_y, pred_y)
    return -r2

In [82]:
from scipy.optimize import differential_evolution
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(2, 300), (0.0001, 0.5), (2, 10)], 
                                          updating='deferred',
                                          workers=10, 
                                          disp=True,
                                          tol=0.00001,
                                          atol=0.00001,
                                          maxiter=1,
                                          args=(X_train_clustered, train_y1, X_test_clustered, test_y1))
print(optimization_res)

differential_evolution step 1: f(x)= -0.853056
Polishing solution with 'L-BFGS-B'
 message: Maximum number of iterations has been exceeded.
 success: False
     fun: -0.8530560292325102
       x: [ 1.818e+02  2.601e-01  2.763e+00]
     nit: 1
    nfev: 370


In [83]:
from metrics import print_metrics
from sklearn.ensemble import GradientBoostingRegressor

# Create an instance of GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor(n_estimators=int(optimization_res.x[0]), learning_rate=optimization_res.x[1], max_depth=int(optimization_res.x[2]), random_state=42)

# Fit the regressor to the training data
gb_regressor.fit(X_train_clustered, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, gb_regressor.predict(X_test_clustered))

print("------ train metrics ------")
print_metrics(train_y1, gb_regressor.predict(X_train_clustered))

print("------ validate metrics ------")
print_metrics(val_y1, gb_regressor.predict(X_val_clustered))

------ test metrics ------
Mean Squared Error (MSE):              23579581024782.3554687500
Root Mean Squared Error (RMSE):        4855881.0760543095
Mean Absolute Error (MAE):             2752210.1142096259
R-squared (R²):                        0.8530560292
Mean Absolute Percentage Error (MAPE): 0.0687117293
Max Error (ME):                        26806958.1641500369
Median Absolute Error (MedAE):         1298936.8345185220
------ train metrics ------
Mean Squared Error (MSE):              9529922458580.2968750000
Root Mean Squared Error (RMSE):        3087057.2489962503
Mean Absolute Error (MAE):             1874872.9661853479
R-squared (R²):                        0.9461262775
Mean Absolute Percentage Error (MAPE): 0.0479801951
Max Error (ME):                        18566170.5566621274
Median Absolute Error (MedAE):         1056265.2274072915
------ validate metrics ------
Mean Squared Error (MSE):              40148966175940.3203125000
Root Mean Squared Error (RMSE):        6336321

In [84]:
X_train_clustered_GBR = np.concatenate((X_train_clustered, gb_regressor.predict(X_train_clustered).reshape(-1, 1)), axis=1)
X_test_clustered_GBR = np.concatenate((X_test_clustered, gb_regressor.predict(X_test_clustered).reshape(-1, 1)), axis=1)
X_val_clustered_GBR = np.concatenate((X_val_clustered, gb_regressor.predict(X_val_clustered).reshape(-1, 1)), axis=1)

<h2> Scale features </h2>

In [85]:
scaler_GBR = MaxAbsScaler()
scaler_GBR.fit(X_train_clustered_GBR)
X_train_clustered_GBR = scaler_GBR.transform(X_train_clustered_GBR)
X_test_clustered_GBR = scaler_GBR.transform(X_test_clustered_GBR)
X_val_clustered_GBR = scaler_GBR.transform(X_val_clustered_GBR)

In [93]:
from sklearn.linear_model import Ridge
from metrics import print_metrics


# Create a Ridge regression model
ridge_reg = Ridge(alpha=2.5)  # You can adjust the regularization strength with the alpha parameter

# Train the model
ridge_reg.fit(X_train_clustered_GBR, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, ridge_reg.predict(X_test_clustered_GBR))

print("------ train metrics ------")
print_metrics(train_y1, ridge_reg.predict(X_train_clustered_GBR))

print("------ val metrics ------")
print_metrics(val_y1, ridge_reg.predict(X_val_clustered_GBR))


------ test metrics ------
Mean Squared Error (MSE):              25176746254857.2070312500
Root Mean Squared Error (RMSE):        5017643.4961899407
Mean Absolute Error (MAE):             2887868.8556013079
R-squared (R²):                        0.8431027650
Mean Absolute Percentage Error (MAPE): 0.0718018654
Max Error (ME):                        26975998.7333675176
Median Absolute Error (MedAE):         1604044.4333808199
------ train metrics ------
Mean Squared Error (MSE):              12897229201348.5546875000
Root Mean Squared Error (RMSE):        3591271.2514301334
Mean Absolute Error (MAE):             2205880.9963348070
R-squared (R²):                        0.9270905141
Mean Absolute Percentage Error (MAPE): 0.0554408496
Max Error (ME):                        19921438.0914819688
Median Absolute Error (MedAE):         1386831.1923695318
------ val metrics ------
Mean Squared Error (MSE):              35702698130993.7968750000
Root Mean Squared Error (RMSE):        5975173.481